In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

class CustomDataset(Dataset):
    def __init__(self, csv_file, method='endpoint'):
        self.data = pd.read_csv(csv_file)
        self.labels = self.data['class'].values
        self.method = method

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        file_path = self.data.iloc[idx]['path']
        label = 0 if self.labels[idx] == 'NoClass' else 1

        raw_embedding = torch.load(file_path)
        if self.method == 'endpoint':
            embedding = torch.cat((raw_embedding[:, 0, :], raw_embedding[:, -1, :]), dim=1)
        elif self.method == 'diff-sum':
            embedding = torch.cat(
                (
                    raw_embedding[:, 0, :] + raw_embedding[:, -1, :],
                    raw_embedding[:, 0, :] - raw_embedding[:, -1, :]
                ),
                dim=1
            )
        elif self.method == 'coherent':
            embedding = torch.cat(
                (
                    raw_embedding[:, 0, :360],
                    raw_embedding[:, -1, 360:720],
                    torch.dot(
                        raw_embedding[:, 0, :].squeeze()[720:],
                        raw_embedding[:, -1, :].squeeze()[720:]
                    ).unsqueeze(0).unsqueeze(0)
                ),
                dim=1
            )
        elif self.method == 'maxpool':
            embedding = torch.max(raw_embedding, dim=1)[0]
        elif self.method == 'avgpool':
            embedding = torch.mean(raw_embedding, dim=1)

        return embedding, torch.FloatTensor([label])

/tmp/ipykernel_50691/2831240186.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import torch.nn.functional as F
import torch.nn as nn

class CustomDeepClassifier(nn.Module):
    def __init__(self, input_dim=1536, hidden_dim=2048):
        super(CustomDeepClassifier, self).__init__()

        layers = []
        layers.append(nn.Linear(input_dim, hidden_dim))
        layers.append(nn.ReLU())
        layers.append(nn.BatchNorm1d(1))

        layers.append(nn.Linear(hidden_dim, hidden_dim))
        layers.append(nn.ReLU())
        layers.append(nn.BatchNorm1d(1))

        layers.append(nn.Linear(hidden_dim, 1))

        self.layers = nn.Sequential(*layers)

    def forward(self, x):
        x = self.layers(x)
        x = F.sigmoid(x)
        return x

In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import precision_recall_fscore_support, classification_report, accuracy_score
import matplotlib.pyplot as plt
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def train_validate_model(model, train_loader, val_loader, num_epochs=10,
                         learning_rate=0.1, step_report=5, stepslr=10, gamma=0.9,
                         sgd=False):
    model.to(device)
    criterion = nn.BCELoss()
    if sgd:
        optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
    else:
        optimizer = optim.AdamW(model.parameters(), lr=learning_rate)
    scheduler = optim.lr_scheduler.StepLR(optimizer, stepslr, gamma=gamma)

    train_losses = []
    val_losses = []
    # precision_scores = []
    # recall_scores = []
    # f1_scores = []
    # accuracy_scores = []

    for epoch in tqdm(range(num_epochs), total=num_epochs):
        model.train()
        running_train_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs.float())
            loss = criterion(outputs.squeeze(-1), labels)
            loss.backward()
            optimizer.step()
            running_train_loss += loss.item()
        
        train_loss = running_train_loss / len(train_loader)
        train_losses.append(train_loss)

        model.eval()
        running_val_loss = 0.0
        all_preds = []
        all_labels = []
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs.float())
                loss = criterion(outputs.squeeze(-1), labels)
                running_val_loss += loss.item()

                predicted = outputs.squeeze(-1)
                all_preds.extend(predicted.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

        val_loss = running_val_loss / len(val_loader)
        val_losses.append(val_loss)

        # precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average='binary')
        # precision_scores.append(precision)
        # recall_scores.append(recall)
        # f1_scores.append(f1)

        # accuracy = accuracy_score(all_labels, all_preds)
        # accuracy_scores.append(accuracy)

        scheduler.step()

        # print(f"Epoch [{epoch + 1}/{num_epochs}] "
        #     f"Train Loss: {train_loss:.4f} "
        #     f"Val Loss: {val_loss:.4f} "
        #     f"Precision: {precision:.4f} "
        #     f"Recall: {recall:.4f} "
        #     f"Accuracy: {accuracy:.4f} "
        #     f"F1 Score: {f1:.4f}\n")
        if (epoch + 1) % step_report == 0:
            print(f"Epoch [{epoch + 1}/{num_epochs}] "
                f"Train Loss: {train_loss:.4f} "
                f"Val Loss: {val_loss:.4f} ")
            print(classification_report(all_labels, [1 if x > 0.5 else 0 for x in all_preds]))

    return train_losses, val_losses#, precision_scores, recall_scores, f1_scores

In [10]:
train_csv_path = 'train.csv'
test_csv_path = 'dev.csv'

method = 'diff-sum'

train_dataset = CustomDataset(train_csv_path, method=method)
test_dataset = CustomDataset(test_csv_path, method=method)

train_batch_size = 256
test_batch_size = 256

train_data_loader = DataLoader(train_dataset, batch_size=train_batch_size, shuffle=True)
test_data_loader = DataLoader(test_dataset, batch_size=test_batch_size, shuffle=False)

model = CustomDeepClassifier(input_dim=train_dataset[0][0].shape[1])
tl, vl = train_validate_model(model,
                              train_data_loader,
                              test_data_loader,
                              num_epochs=100,
                              learning_rate=3e-2,
                              stepslr=10,
                              gamma=0.5,
                              sgd=True)

torch.save(model.state_dict(), f'binary_model_{method}.pt')

  0%|          | 0/100 [00:00<?, ?it/s]

  5%|▌         | 5/100 [04:29<1:24:53, 53.61s/it]

Epoch [5/100] Train Loss: 0.2806 Val Loss: 0.3338 
              precision    recall  f1-score   support

         0.0       0.93      0.80      0.86      1801
         1.0       0.82      0.94      0.88      1750

    accuracy                           0.87      3551
   macro avg       0.88      0.87      0.87      3551
weighted avg       0.88      0.87      0.87      3551



 10%|█         | 10/100 [08:47<1:17:35, 51.72s/it]

Epoch [10/100] Train Loss: 0.1884 Val Loss: 0.2660 
              precision    recall  f1-score   support

         0.0       0.92      0.86      0.89      1801
         1.0       0.86      0.92      0.89      1750

    accuracy                           0.89      3551
   macro avg       0.89      0.89      0.89      3551
weighted avg       0.89      0.89      0.89      3551



...

Epoch [90/100] Train Loss: 0.0011 Val Loss: 0.7767 
              precision    recall  f1-score   support

         0.0       0.89      0.91      0.90      1801
         1.0       0.90      0.88      0.89      1750

    accuracy                           0.89      3551
   macro avg       0.89      0.89      0.89      3551
weighted avg       0.89      0.89      0.89      3551



 95%|█████████▌| 95/100 [1:17:57<04:12, 50.43s/it]

Epoch [95/100] Train Loss: 0.0011 Val Loss: 0.7833 
              precision    recall  f1-score   support

         0.0       0.89      0.91      0.90      1801
         1.0       0.90      0.88      0.89      1750

    accuracy                           0.89      3551
   macro avg       0.89      0.89      0.89      3551
weighted avg       0.89      0.89      0.89      3551



100%|██████████| 100/100 [1:22:09<00:00, 49.29s/it]

Epoch [100/100] Train Loss: 0.0010 Val Loss: 0.7929 
              precision    recall  f1-score   support

         0.0       0.89      0.91      0.90      1801
         1.0       0.90      0.88      0.89      1750

    accuracy                           0.89      3551
   macro avg       0.89      0.89      0.89      3551
weighted avg       0.89      0.89      0.89      3551



In [12]:
train_csv_path = 'train.csv'
test_csv_path = 'dev.csv'

method = 'diff-sum'

train_dataset = CustomDataset(train_csv_path, method=method)
test_dataset = CustomDataset(test_csv_path, method=method)

train_batch_size = 256
test_batch_size = 256

train_data_loader = DataLoader(train_dataset, batch_size=train_batch_size, shuffle=True)
test_data_loader = DataLoader(test_dataset, batch_size=test_batch_size, shuffle=False)

model = CustomDeepClassifier(input_dim=train_dataset[0][0].shape[1])
tl, vl = train_validate_model(model,
                              train_data_loader,
                              test_data_loader,
                              num_epochs=30,
                              learning_rate=3e-4,
                              stepslr=1,
                              gamma=0.8,
                              sgd=True,
                              step_report=1)

torch.save(model.state_dict(), f'binary_model_{method}_1.pt')

  0%|          | 0/30 [00:00<?, ?it/s]

  3%|▎         | 1/30 [00:54<26:17, 54.40s/it]

Epoch [1/30] Train Loss: 0.5824 Val Loss: 0.4848 
              precision    recall  f1-score   support

         0.0       0.77      0.84      0.80      1801
         1.0       0.82      0.74      0.77      1750

    accuracy                           0.79      3551
   macro avg       0.79      0.79      0.79      3551
weighted avg       0.79      0.79      0.79      3551



  7%|▋         | 2/30 [01:48<25:24, 54.44s/it]

Epoch [2/30] Train Loss: 0.4410 Val Loss: 0.4113 
              precision    recall  f1-score   support

         0.0       0.83      0.82      0.82      1801
         1.0       0.82      0.83      0.82      1750

    accuracy                           0.82      3551
   macro avg       0.82      0.82      0.82      3551
weighted avg       0.82      0.82      0.82      3551



...

Epoch [28/30] Train Loss: 0.2847 Val Loss: 0.3270 
              precision    recall  f1-score   support

         0.0       0.86      0.87      0.87      1801
         1.0       0.87      0.85      0.86      1750

    accuracy                           0.86      3551
   macro avg       0.86      0.86      0.86      3551
weighted avg       0.86      0.86      0.86      3551



 97%|█████████▋| 29/30 [24:26<00:49, 49.87s/it]

Epoch [29/30] Train Loss: 0.2855 Val Loss: 0.3270 
              precision    recall  f1-score   support

         0.0       0.86      0.87      0.87      1801
         1.0       0.87      0.85      0.86      1750

    accuracy                           0.86      3551
   macro avg       0.86      0.86      0.86      3551
weighted avg       0.86      0.86      0.86      3551



100%|██████████| 30/30 [25:16<00:00, 50.53s/it]

Epoch [30/30] Train Loss: 0.2862 Val Loss: 0.3270 
              precision    recall  f1-score   support

         0.0       0.86      0.87      0.87      1801
         1.0       0.87      0.85      0.86      1750

    accuracy                           0.86      3551
   macro avg       0.86      0.86      0.86      3551
weighted avg       0.86      0.86      0.86      3551



In [17]:
train_csv_path = 'train.csv'
test_csv_path = 'dev.csv'

method = 'endpoint'

train_dataset = CustomDataset(train_csv_path, method=method)
test_dataset = CustomDataset(test_csv_path, method=method)

train_batch_size = 256
test_batch_size = 256

train_data_loader = DataLoader(train_dataset, batch_size=train_batch_size, shuffle=True)
test_data_loader = DataLoader(test_dataset, batch_size=test_batch_size, shuffle=False)

model = CustomDeepClassifier(input_dim=train_dataset[0][0].shape[1])
tl, vl = train_validate_model(model,
                              train_data_loader,
                              test_data_loader,
                              num_epochs=300,
                              learning_rate=3e-4,
                              stepslr=30,
                              gamma=0.75)

torch.save(model.state_dict(), f'binary_model_{method}.pt')

 10%|█         | 30/300 [19:26<2:57:19, 39.40s/it]

[array([0.9992539], dtype=float32), array([0.9992539], dtype=float32), array([0.99271566], dtype=float32), array([0.999933], dtype=float32), array([0.9996593], dtype=float32), array([0.99999094], dtype=float32), array([0.9979373], dtype=float32), array([0.90290123], dtype=float32), array([1.], dtype=float32), array([0.9867425], dtype=float32), array([0.9999107], dtype=float32), array([0.99969494], dtype=float32), array([0.9986078], dtype=float32), array([0.9992507], dtype=float32), array([0.9998908], dtype=float32), array([0.999966], dtype=float32), array([0.9977102], dtype=float32), array([0.9999993], dtype=float32), array([0.88852036], dtype=float32), array([0.9999989], dtype=float32), array([0.9997731], dtype=float32), array([0.9984864], dtype=float32), array([0.1293112], dtype=float32), array([0.3460961], dtype=float32), array([0.9999759], dtype=float32), array([0.9999982], dtype=float32), array([0.81751966], dtype=float32), array([0.99999976], dtype=float32), array([0.9998522], dt

 20%|██        | 60/300 [38:28<2:41:49, 40.46s/it]

[array([0.99991345], dtype=float32), array([0.99991345], dtype=float32), array([0.99555284], dtype=float32), array([0.99999046], dtype=float32), array([0.9999566], dtype=float32), array([0.9999982], dtype=float32), array([0.99947923], dtype=float32), array([0.88321626], dtype=float32), array([1.], dtype=float32), array([0.9951108], dtype=float32), array([0.99999356], dtype=float32), array([0.9998776], dtype=float32), array([0.99958235], dtype=float32), array([0.9997868], dtype=float32), array([0.99997973], dtype=float32), array([0.99999785], dtype=float32), array([0.99977857], dtype=float32), array([1.], dtype=float32), array([0.9613548], dtype=float32), array([0.9999999], dtype=float32), array([0.9999833], dtype=float32), array([0.9997476], dtype=float32), array([0.16654086], dtype=float32), array([0.69695824], dtype=float32), array([0.9999887], dtype=float32), array([0.9999995], dtype=float32), array([0.90840244], dtype=float32), array([1.], dtype=float32), array([0.9996253], dtype=f

 30%|███       | 90/300 [57:26<2:17:34, 39.31s/it]

...

[array([1.], dtype=float32), array([1.], dtype=float32), array([0.7840277], dtype=float32), array([0.99999976], dtype=float32), array([0.99977], dtype=float32), array([1.], dtype=float32), array([0.99997795], dtype=float32), array([0.9035791], dtype=float32), array([0.9999999], dtype=float32), array([0.07053714], dtype=float32), array([0.99995136], dtype=float32), array([0.95482], dtype=float32), array([0.9965995], dtype=float32), array([0.9999716], dtype=float32), array([0.9999976], dtype=float32), array([1.], dtype=float32), array([0.9999212], dtype=float32), array([1.], dtype=float32), array([0.54129416], dtype=float32), array([0.9999999], dtype=float32), array([0.99999845], dtype=float32), array([0.9999237], dtype=float32), array([0.03742378], dtype=float32), array([0.04207102], dtype=float32), array([1.], dtype=float32), array([0.99999905], dtype=float32), array([0.99196106], dtype=float32), array([1.], dtype=float32), array([0.99999976], dtype=float32), array([0.99999917], dtype=

 90%|█████████ | 270/300 [2:52:19<19:47, 39.57s/it]

[array([0.9999999], dtype=float32), array([0.9999999], dtype=float32), array([0.9432296], dtype=float32), array([0.9999995], dtype=float32), array([0.9991844], dtype=float32), array([1.], dtype=float32), array([0.99954057], dtype=float32), array([0.991433], dtype=float32), array([1.], dtype=float32), array([0.90663564], dtype=float32), array([0.9999974], dtype=float32), array([0.9996773], dtype=float32), array([0.99999464], dtype=float32), array([0.9998135], dtype=float32), array([0.99986744], dtype=float32), array([1.], dtype=float32), array([0.99947315], dtype=float32), array([1.], dtype=float32), array([0.14409569], dtype=float32), array([1.], dtype=float32), array([0.999995], dtype=float32), array([0.9999925], dtype=float32), array([0.41795278], dtype=float32), array([0.67281675], dtype=float32), array([0.9999999], dtype=float32), array([0.9999999], dtype=float32), array([0.7721881], dtype=float32), array([1.], dtype=float32), array([0.9999527], dtype=float32), array([0.99988925], 

100%|██████████| 300/300 [3:11:21<00:00, 38.27s/it]

[array([1.], dtype=float32), array([1.], dtype=float32), array([0.94016486], dtype=float32), array([1.], dtype=float32), array([0.99973327], dtype=float32), array([1.], dtype=float32), array([0.99987185], dtype=float32), array([0.99076754], dtype=float32), array([1.], dtype=float32), array([0.9075767], dtype=float32), array([0.9999994], dtype=float32), array([0.9998242], dtype=float32), array([0.99999833], dtype=float32), array([0.99994576], dtype=float32), array([0.99994683], dtype=float32), array([1.], dtype=float32), array([0.9999318], dtype=float32), array([1.], dtype=float32), array([0.14032556], dtype=float32), array([1.], dtype=float32), array([0.9999993], dtype=float32), array([0.99999857], dtype=float32), array([0.41030252], dtype=float32), array([0.7116566], dtype=float32), array([1.], dtype=float32), array([1.], dtype=float32), array([0.8812498], dtype=float32), array([1.], dtype=float32), array([0.9999975], dtype=float32), array([0.999974], dtype=float32), array([1.], dtype

In [22]:
train_csv_path = 'train.csv'
test_csv_path = 'dev.csv'

method = 'endpoint'

train_dataset = CustomDataset(train_csv_path, method=method)
test_dataset = CustomDataset(test_csv_path, method=method)

train_batch_size = 256
test_batch_size = 256

train_data_loader = DataLoader(train_dataset, batch_size=train_batch_size, shuffle=True)
test_data_loader = DataLoader(test_dataset, batch_size=test_batch_size, shuffle=False)

model = CustomDeepClassifier(input_dim=train_dataset[0][0].shape[1])
tl, vl = train_validate_model(model,
                              train_data_loader,
                              test_data_loader,
                              num_epochs=20,
                              learning_rate=3e-4,
                              stepslr=1,
                              gamma=0.95)

torch.save(model.state_dict(), f'binary_model_{method}_1.pt')

  5%|▌         | 1/20 [00:46<14:52, 46.97s/it]

Epoch [1/20] Train Loss: 0.4504 Val Loss: 0.3041 
              precision    recall  f1-score   support

         0.0       0.85      0.91      0.88      1801
         1.0       0.90      0.83      0.87      1750

    accuracy                           0.87      3551
   macro avg       0.88      0.87      0.87      3551
weighted avg       0.88      0.87      0.87      3551



 10%|█         | 2/20 [01:33<13:56, 46.47s/it]

Epoch [2/20] Train Loss: 0.1971 Val Loss: 0.2698 
              precision    recall  f1-score   support

         0.0       0.92      0.86      0.89      1801
         1.0       0.86      0.93      0.89      1750

    accuracy                           0.89      3551
   macro avg       0.89      0.89      0.89      3551
weighted avg       0.89      0.89      0.89      3551



 15%|█▌        | 3/20 [02:19<13:10, 46.51s/it]

...

Epoch [18/20] Train Loss: 0.0017 Val Loss: 0.4117 
              precision    recall  f1-score   support

         0.0       0.89      0.91      0.90      1801
         1.0       0.90      0.88      0.89      1750

    accuracy                           0.90      3551
   macro avg       0.90      0.90      0.90      3551
weighted avg       0.90      0.90      0.90      3551



 95%|█████████▌| 19/20 [14:47<00:47, 47.40s/it]

Epoch [19/20] Train Loss: 0.0018 Val Loss: 0.4328 
              precision    recall  f1-score   support

         0.0       0.90      0.90      0.90      1801
         1.0       0.89      0.90      0.90      1750

    accuracy                           0.90      3551
   macro avg       0.90      0.90      0.90      3551
weighted avg       0.90      0.90      0.90      3551



100%|██████████| 20/20 [15:34<00:00, 46.73s/it]

Epoch [20/20] Train Loss: 0.0018 Val Loss: 0.4410 
              precision    recall  f1-score   support

         0.0       0.89      0.91      0.90      1801
         1.0       0.90      0.89      0.89      1750

    accuracy                           0.90      3551
   macro avg       0.90      0.90      0.90      3551
weighted avg       0.90      0.90      0.90      3551



In [25]:
train_csv_path = 'train.csv'
test_csv_path = 'dev.csv'

method = 'diff-sum'

train_dataset = CustomDataset(train_csv_path, method=method)
test_dataset = CustomDataset(test_csv_path, method=method)

train_batch_size = 256
test_batch_size = 256

train_data_loader = DataLoader(train_dataset, batch_size=train_batch_size, shuffle=True)
test_data_loader = DataLoader(test_dataset, batch_size=test_batch_size, shuffle=False)

model = CustomDeepClassifier(input_dim=train_dataset[0][0].shape[1])
tl, vl = train_validate_model(model,
                              train_data_loader,
                              test_data_loader,
                              num_epochs=100,
                              learning_rate=3e-4,
                              stepslr=10,
                              gamma=0.5)

torch.save(model.state_dict(), f'binary_model_{method}.pt')

 10%|█         | 10/100 [06:48<1:02:56, 41.97s/it]

Epoch [10/100] Train Loss: 0.0021 Val Loss: 0.3933 
              precision    recall  f1-score   support

         0.0       0.91      0.91      0.91      1801
         1.0       0.91      0.90      0.90      1750

    accuracy                           0.91      3551
   macro avg       0.91      0.91      0.91      3551
weighted avg       0.91      0.91      0.91      3551



 20%|██        | 20/100 [13:35<55:43, 41.80s/it]  

Epoch [20/100] Train Loss: 0.0013 Val Loss: 0.4417 
              precision    recall  f1-score   support

         0.0       0.91      0.91      0.91      1801
         1.0       0.90      0.91      0.91      1750

    accuracy                           0.91      3551
   macro avg       0.91      0.91      0.91      3551
weighted avg       0.91      0.91      0.91      3551



 30%|███       | 30/100 [20:11<47:09, 40.43s/it]

...

Epoch [80/100] Train Loss: 0.0010 Val Loss: 0.7320 
              precision    recall  f1-score   support

         0.0       0.91      0.91      0.91      1801
         1.0       0.91      0.90      0.90      1750

    accuracy                           0.91      3551
   macro avg       0.91      0.91      0.91      3551
weighted avg       0.91      0.91      0.91      3551



 90%|█████████ | 90/100 [58:34<06:22, 38.20s/it]

Epoch [90/100] Train Loss: 0.0017 Val Loss: 0.4955 
              precision    recall  f1-score   support

         0.0       0.89      0.91      0.90      1801
         1.0       0.90      0.89      0.90      1750

    accuracy                           0.90      3551
   macro avg       0.90      0.90      0.90      3551
weighted avg       0.90      0.90      0.90      3551



100%|██████████| 100/100 [1:04:49<00:00, 38.90s/it]

Epoch [100/100] Train Loss: 0.0011 Val Loss: 0.5265 
              precision    recall  f1-score   support

         0.0       0.89      0.92      0.90      1801
         1.0       0.91      0.88      0.90      1750

    accuracy                           0.90      3551
   macro avg       0.90      0.90      0.90      3551
weighted avg       0.90      0.90      0.90      3551



In [26]:
train_csv_path = 'train.csv'
test_csv_path = 'dev.csv'

method = 'coherent'

train_dataset = CustomDataset(train_csv_path, method=method)
test_dataset = CustomDataset(test_csv_path, method=method)

train_batch_size = 256
test_batch_size = 256

train_data_loader = DataLoader(train_dataset, batch_size=train_batch_size, shuffle=True)
test_data_loader = DataLoader(test_dataset, batch_size=test_batch_size, shuffle=False)

model = CustomDeepClassifier(input_dim=train_dataset[0][0].shape[1])
tl, vl = train_validate_model(model,
                              train_data_loader,
                              test_data_loader,
                              num_epochs=100,
                              learning_rate=3e-4,
                              stepslr=10,
                              gamma=0.5)

torch.save(model.state_dict(), f'binary_model_{method}.pt')

 10%|█         | 10/100 [05:42<53:00, 35.34s/it]

Epoch [10/100] Train Loss: 0.0117 Val Loss: 0.5348 
              precision    recall  f1-score   support

         0.0       0.87      0.88      0.88      1801
         1.0       0.88      0.87      0.87      1750

    accuracy                           0.87      3551
   macro avg       0.88      0.87      0.87      3551
weighted avg       0.88      0.87      0.87      3551



 20%|██        | 20/100 [11:23<47:05, 35.32s/it]

Epoch [20/100] Train Loss: 0.0087 Val Loss: 0.6511 
              precision    recall  f1-score   support

         0.0       0.89      0.87      0.88      1801
         1.0       0.87      0.89      0.88      1750

    accuracy                           0.88      3551
   macro avg       0.88      0.88      0.88      3551
weighted avg       0.88      0.88      0.88      3551



 30%|███       | 30/100 [16:59<40:11, 34.44s/it]

...

Epoch [80/100] Train Loss: 0.0012 Val Loss: 1.0238 
              precision    recall  f1-score   support

         0.0       0.86      0.89      0.87      1801
         1.0       0.88      0.85      0.87      1750

    accuracy                           0.87      3551
   macro avg       0.87      0.87      0.87      3551
weighted avg       0.87      0.87      0.87      3551



 90%|█████████ | 90/100 [53:28<06:18, 37.83s/it]

Epoch [90/100] Train Loss: 0.0010 Val Loss: 1.0348 
              precision    recall  f1-score   support

         0.0       0.86      0.89      0.87      1801
         1.0       0.89      0.85      0.87      1750

    accuracy                           0.87      3551
   macro avg       0.87      0.87      0.87      3551
weighted avg       0.87      0.87      0.87      3551



100%|██████████| 100/100 [59:33<00:00, 35.74s/it]

Epoch [100/100] Train Loss: 0.0011 Val Loss: 1.1077 
              precision    recall  f1-score   support

         0.0       0.86      0.89      0.87      1801
         1.0       0.88      0.85      0.87      1750

    accuracy                           0.87      3551
   macro avg       0.87      0.87      0.87      3551
weighted avg       0.87      0.87      0.87      3551



In [27]:
train_csv_path = 'train.csv'
test_csv_path = 'dev.csv'

method = 'maxpool'

train_dataset = CustomDataset(train_csv_path, method=method)
test_dataset = CustomDataset(test_csv_path, method=method)

train_batch_size = 256
test_batch_size = 256

train_data_loader = DataLoader(train_dataset, batch_size=train_batch_size, shuffle=True)
test_data_loader = DataLoader(test_dataset, batch_size=test_batch_size, shuffle=False)

model = CustomDeepClassifier(input_dim=train_dataset[0][0].shape[1])
tl, vl = train_validate_model(model,
                              train_data_loader,
                              test_data_loader,
                              num_epochs=100,
                              learning_rate=3e-4,
                              stepslr=10,
                              gamma=0.5)

torch.save(model.state_dict(), f'binary_model_{method}.pt')

 10%|█         | 10/100 [07:41<1:11:07, 47.42s/it]

Epoch [10/100] Train Loss: 0.2929 Val Loss: 0.7236 
              precision    recall  f1-score   support

         0.0       0.64      0.96      0.77      1801
         1.0       0.91      0.44      0.59      1750

    accuracy                           0.70      3551
   macro avg       0.77      0.70      0.68      3551
weighted avg       0.77      0.70      0.68      3551



 20%|██        | 20/100 [15:38<1:04:50, 48.63s/it]

Epoch [20/100] Train Loss: 0.1366 Val Loss: 0.8831 
              precision    recall  f1-score   support

         0.0       0.68      0.90      0.78      1801
         1.0       0.85      0.57      0.68      1750

    accuracy                           0.74      3551
   macro avg       0.77      0.74      0.73      3551
weighted avg       0.76      0.74      0.73      3551



 30%|███       | 30/100 [23:31<56:57, 48.82s/it]  

...

Epoch [80/100] Train Loss: 0.0282 Val Loss: 2.7582 
              precision    recall  f1-score   support

         0.0       0.74      0.74      0.74      1801
         1.0       0.73      0.73      0.73      1750

    accuracy                           0.73      3551
   macro avg       0.73      0.73      0.73      3551
weighted avg       0.73      0.73      0.73      3551



 90%|█████████ | 90/100 [1:11:01<08:09, 48.97s/it]

Epoch [90/100] Train Loss: 0.0418 Val Loss: 2.4277 
              precision    recall  f1-score   support

         0.0       0.70      0.85      0.77      1801
         1.0       0.80      0.63      0.70      1750

    accuracy                           0.74      3551
   macro avg       0.75      0.74      0.73      3551
weighted avg       0.75      0.74      0.73      3551



100%|██████████| 100/100 [1:18:56<00:00, 47.36s/it]

Epoch [100/100] Train Loss: 0.0277 Val Loss: 2.8455 
              precision    recall  f1-score   support

         0.0       0.74      0.76      0.75      1801
         1.0       0.74      0.72      0.73      1750

    accuracy                           0.74      3551
   macro avg       0.74      0.74      0.74      3551
weighted avg       0.74      0.74      0.74      3551



In [28]:
train_csv_path = 'train.csv'
test_csv_path = 'dev.csv'

method = 'avgpool'

train_dataset = CustomDataset(train_csv_path, method=method)
test_dataset = CustomDataset(test_csv_path, method=method)

train_batch_size = 256
test_batch_size = 256

train_data_loader = DataLoader(train_dataset, batch_size=train_batch_size, shuffle=True)
test_data_loader = DataLoader(test_dataset, batch_size=test_batch_size, shuffle=False)

model = CustomDeepClassifier(input_dim=train_dataset[0][0].shape[1])
tl, vl = train_validate_model(model,
                              train_data_loader,
                              test_data_loader,
                              num_epochs=100,
                              learning_rate=3e-4,
                              stepslr=10,
                              gamma=0.5)

torch.save(model.state_dict(), f'binary_model_{method}.pt')

 10%|█         | 10/100 [06:15<58:00, 38.67s/it] 

Epoch [10/100] Train Loss: 0.0636 Val Loss: 0.8119 
              precision    recall  f1-score   support

         0.0       0.75      0.88      0.81      1801
         1.0       0.85      0.70      0.77      1750

    accuracy                           0.79      3551
   macro avg       0.80      0.79      0.79      3551
weighted avg       0.80      0.79      0.79      3551



 20%|██        | 20/100 [12:31<51:32, 38.66s/it]

Epoch [20/100] Train Loss: 0.0230 Val Loss: 0.9662 
              precision    recall  f1-score   support

         0.0       0.82      0.80      0.81      1801
         1.0       0.80      0.82      0.81      1750

    accuracy                           0.81      3551
   macro avg       0.81      0.81      0.81      3551
weighted avg       0.81      0.81      0.81      3551



 30%|███       | 30/100 [18:46<45:11, 38.74s/it]

...

Epoch [80/100] Train Loss: 0.0038 Val Loss: 1.7775 
              precision    recall  f1-score   support

         0.0       0.79      0.80      0.80      1801
         1.0       0.79      0.78      0.79      1750

    accuracy                           0.79      3551
   macro avg       0.79      0.79      0.79      3551
weighted avg       0.79      0.79      0.79      3551



 90%|█████████ | 90/100 [56:28<06:27, 38.75s/it]

Epoch [90/100] Train Loss: 0.0031 Val Loss: 1.8941 
              precision    recall  f1-score   support

         0.0       0.75      0.87      0.80      1801
         1.0       0.84      0.70      0.76      1750

    accuracy                           0.78      3551
   macro avg       0.79      0.78      0.78      3551
weighted avg       0.79      0.78      0.78      3551



100%|██████████| 100/100 [1:02:43<00:00, 37.63s/it]

Epoch [100/100] Train Loss: 0.0068 Val Loss: 1.9716 
              precision    recall  f1-score   support

         0.0       0.81      0.79      0.80      1801
         1.0       0.79      0.81      0.80      1750

    accuracy                           0.80      3551
   macro avg       0.80      0.80      0.80      3551
weighted avg       0.80      0.80      0.80      3551

